# <center> Идентификация пользователей по посещенным веб-страницам
<img src='http://i.istockimg.com/file_thumbview_approve/21546327/5/stock-illustration-21546327-identification-de-l-utilisateur.jpg'>

# <center>Неделя 5.  Соревнование Kaggle "Identify Me If You Can"

На этой неделе мы вспомним про концепцию стохастического градиентного спуска и опробуем классификатор Scikit-learn SGDClassifier, который работает намного быстрее на больших выборках, чем алгоритмы, которые мы тестировали на 4 неделе. Также мы познакомимся с данными [соревнования](https://inclass.kaggle.com/c/identify-me-if-you-can-yandex-mipt/) Kaggle по идентификации пользователей и сделаем в нем первые посылки. По итогам этой недели дополнительные баллы получат те, кто попадет в топ-30 публичного лидерборда соревнования.

**В этой части проекта Вам могут быть полезны видеозаписи следующих лекций курса "Обучение на размеченных данных":**
   - [Стохатический градиентный спуск](https://www.coursera.org/learn/supervised-learning/lecture/xRY50/stokhastichieskii-ghradiientnyi-spusk)
   - [Линейные модели. Sklearn.linear_model. Классификация](https://www.coursera.org/learn/supervised-learning/lecture/EBg9t/linieinyie-modieli-sklearn-linear-model-klassifikatsiia)
   
**Также рекомендуется вернуться и просмотреть [задание](https://www.coursera.org/learn/supervised-learning/programming/t2Idc/linieinaia-rieghriessiia-i-stokhastichieskii-ghradiientnyi-spusk) "Линейная регрессия и стохастический градиентный спуск" 1 недели 2 курса специализации.**

In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, f1_score

**Считаем данные [соревнования](https://inclass.kaggle.com/c/identify-me-if-you-can-yandex-mipt/data) в DataFrame train_df и test_df (обучающая и тестовая выборки).**

In [2]:
train_df = pd.read_csv('kaggle_data/train_sessions.csv', index_col='session_id')
test_df = pd.read_csv('kaggle_data/test_sessions.csv', index_col='session_id')

In [3]:
train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,user_id
session_id,,,,,,,,,,,,,,,,,,,,,
1,8,2014-01-04 08:44:50,11.0,2014-01-04 08:44:50,82.0,2014-01-04 08:45:19,68.0,2014-01-04 08:45:25,8.0,2014-01-04 08:45:25,...,2014-01-04 08:45:51,8403.0,2014-01-04 08:45:51,932.0,2014-01-04 08:45:53,3260.0,2014-01-04 08:45:53,8.0,2014-01-04 08:45:53,1845
2,111,2014-03-18 10:33:20,78.0,2014-03-18 10:33:31,151.0,2014-03-18 10:33:31,111.0,2014-03-18 10:33:31,1401.0,2014-03-18 10:33:31,...,2014-03-18 10:33:32,1375.0,2014-03-18 10:33:32,38.0,2014-03-18 10:33:32,1401.0,2014-03-18 10:33:32,97.0,2014-03-18 10:33:34,3322
3,11,2014-12-02 13:13:41,3187.0,2014-12-02 13:13:41,132.0,2014-12-02 13:13:42,496.0,2014-12-02 13:13:42,1969.0,2014-12-02 13:13:45,...,2014-12-02 13:13:45,3187.0,2014-12-02 13:13:45,82.0,2014-12-02 13:13:46,3191.0,2014-12-02 13:13:46,3184.0,2014-12-02 13:13:47,2003
4,668,2014-02-14 15:16:45,1965.0,2014-02-14 15:17:13,598.0,2014-02-14 15:20:47,1965.0,2014-02-14 15:21:13,284.0,2014-02-14 15:21:14,...,2014-02-14 15:21:14,38.0,2014-02-14 15:21:14,4451.0,2014-02-14 15:21:14,4537.0,2014-02-14 15:21:15,11.0,2014-02-14 15:21:15,1373
5,1943,2014-03-17 15:19:40,1943.0,2014-03-17 15:20:10,1943.0,2014-03-17 15:21:40,1943.0,2014-03-17 15:22:10,1943.0,2014-03-17 15:22:39,...,2014-03-17 15:22:39,1952.0,2014-03-17 15:22:41,1943.0,2014-03-17 15:22:41,1943.0,2014-03-17 15:22:42,1943.0,2014-03-17 15:22:43,1737


**Объединим обучающую и тестовую выборки – это понадобится, чтоб вместе потом привести их к разреженному формату.**

In [4]:
train_test_df = pd.concat([train_df, test_df])

In [5]:
train_df.shape, test_df.shape, train_test_df.shape

((95319, 21), (41177, 20), (136496, 21))

В обучающей выборке видим следующие признаки:
    - site1 – индекс первого посещенного сайта в сессии
    - time1 – время посещения первого сайта в сессии
    - ...
    - site10 – индекс 10-го посещенного сайта в сессии
    - time10 – время посещения 10-го сайта в сессии
    - user_id – ID пользователя
    
Сессии пользователей выделены таким образом, что они не могут быть длинее получаса или 10 сайтов. То есть сессия считается оконченной либо когда пользователь посетил 10 сайтов подряд, либо когда сессия заняла по времени более 30 минут. 

**Посмотрим на статистику признаков.**

Пропуски возникают там, где сессии короткие (менее 10 сайтов). Скажем, если человек 1 января 2015 года посетил *vk.com* в 20:01, потом *yandex.ru* в 20:29, затем *google.com* в 20:33, то первая его сессия будет состоять только из двух сайтов (site1 – ID сайта *vk.com*, time1 – 2015-01-01 20:01:00, site2 – ID сайта  *yandex.ru*, time2 – 2015-01-01 20:29:00, остальные признаки – NaN), а начиная с *google.com* пойдет новая сессия, потому что уже прошло более 30 минут с момента посещения *vk.com*.

**В обучающей выборке – 550 пользователей.**

In [6]:
train_df['user_id'].nunique()

550

**Пока для прогноза ID пользователя будем использовать только индексы посещенных сайтов. Индексы нумеровались с 1, так что заменим пропуски на нули.**

In [7]:
train_test_df_sites = train_test_df[['site1', 'site2', 'site3', 
                                     'site4','site5', 
                                     'site6','site7', 'site8', 
                                     'site9', 'site10']].fillna(0).astype('int')

In [8]:
train_test_df_sites.head(10)

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
1,8,11,82,68,8,8393,8403,932,3260,8
2,111,78,151,111,1401,151,1375,38,1401,97
3,11,3187,132,496,1969,504,3187,82,3191,3184
4,668,1965,598,1965,284,668,38,4451,4537,11
5,1943,1943,1943,1943,1943,1952,1952,1943,1943,1943
6,2941,2951,2958,2993,162,2967,85,7,2967,2967
7,9996,9996,307,9996,280,9996,9996,307,9996,9996
8,19823,19823,1510,32,19808,8,567,567,654,567
9,72,0,0,0,0,0,0,0,0,0


#### создадим признаки связанные с временем

In [9]:
train_test_df_time = train_test_df[['time1', 'time2', 'time3', 
                                     'time4','time5', 'time6',
                                     'time7','time8', 'time9', 
                                     'time10']].fillna(np.datetime64('nat'))

In [10]:
train_test_df_time = train_test_df_time.apply(pd.to_datetime).values

In [11]:
train_test_df_time.shape

(136496L, 10L)

In [12]:
# время на сайте в сек
train_test_df_time_diff = np.diff(train_test_df_time, axis =1)/np.timedelta64(3, 's')

In [13]:
train_test_df_time_diff = np.nan_to_num(train_test_df_time_diff) # заменим пропуски на ноль

- отмаштабируем признаки

In [14]:
from sklearn.preprocessing import StandardScaler
#tr_ts_df_time_scaled = StandardScaler().fit_transform(train_test_df_time_diff)

In [15]:
# время сессии в 3-х сек
session_timespan = (np.max(train_test_df_time, axis =1)
                                          -np.min(train_test_df_time, axis =1))/np.timedelta64(3, 's')

session_timespan = session_timespan.reshape(train_test_df_time.shape[0],1)

In [16]:
session_timespan.max(), session_timespan.min()

(600.0, 0.0)

In [17]:
session_timespan_scaled = StandardScaler().fit_transform(session_timespan) # время сессии отмаштабированное

In [18]:
# время начала сессии в часах
start_hour = (pd.to_datetime(np.min(train_test_df_time, axis =1))).hour
start_hour = start_hour.reshape(train_test_df_time.shape[0],1)

- произведем кодирование временных признаков в категориальные

In [19]:
from sklearn.preprocessing import OneHotEncoder
tr_ts_df_time_encode = OneHotEncoder().fit_transform(train_test_df_time_diff) # перевод категориальных признаков в двоичные

In [20]:
session_timespan_encode = OneHotEncoder().fit_transform(session_timespan)
session_timespan_encode.shape, tr_ts_df_time_encode.shape

((136496, 601), (136496, 4594))

In [21]:
# начало сессии в часах
start_hour_encod = OneHotEncoder().fit_transform(start_hour) # перевод категориальных признаков в двоичные

In [22]:
# начало сессии по дням недели
day_of_week = (pd.to_datetime(np.min(train_test_df_time, axis =1))).dayofweek
day_of_week = day_of_week.reshape(train_test_df_time.shape[0],1) 

In [23]:
day_of_week_encod = OneHotEncoder().fit_transform(day_of_week) # перевод категориальных признаков в двоичные

In [24]:
session_timespan_encode.shape, start_hour_encod.shape,  day_of_week_encod.shape

((136496, 601), (136496, 17), (136496, 7))

**Создайте разреженные матрицы *X_train_sparse* и *X_test_sparse* аналогично тому, как мы это делали ранее. Используйте объединенную матрицу train_test_df_sites – потом разделите обратно на обучающую и тестовую части.**


**Выделите в отдельный вектор *y* ответы на обучающей выборке.**

In [25]:
def matrix_to_sparse_matrix (matrix):
    """переводим обычную матрицу в разреженноу матрицу 
    где 
    номер столбца это уникальное число из исходной матрицы от 1  до максимального
    значение в строке это сколько раз уникальное число встречалось в строке оригинальной матрицы"""
    import numpy as np
    from scipy.sparse import csr_matrix
        
    NMZ = np.prod(np.array(matrix.shape)) # колличество элементов в matrix
    data = np.array([1]*NMZ)
    indptr = np.arange(0, NMZ+matrix.shape[1], matrix.shape[1])
    return csr_matrix((data, matrix.reshape(-1), indptr), dtype=int)[:,1:]

In [26]:
X_train_test_sparse =  matrix_to_sparse_matrix(train_test_df_sites.values)

In [27]:
X_train_test_sparse.shape#, matrix_time.shape

(136496, 24052)

** добавим к признакам посещенные сайты временные: начало сессии в часах и день недели**

In [28]:
from scipy.sparse import hstack
X_train_test_sparse_time = hstack((X_train_test_sparse, start_hour_encod, day_of_week_encod))#, tr_ts_df_time_encode))
X_train_test_sparse_time.shape

(136496, 24076)

- разделим train , test данные

In [ ]:
X_train_sparse = X_train_test_sparse[: 95319, :]
X_test_sparse = X_train_test_sparse[95319 :, :]
y = train_df['user_id'].values

In [30]:
X_train_sparse.shape, X_test_sparse.shape

((95319, 24076), (41177, 24076))

**Сохраним в pickle-файлы объекты *X_train_sparse*, *X_test_sparse* и *y* (последний – в файл *kaggle_data/train_target.pkl*).**

In [32]:
import pickle
with open('kaggle_data/X_train_sparse.pkl', 'wb') as X_train_sparse_pkl:
    pickle.dump(X_train_sparse, X_train_sparse_pkl)
with open('kaggle_data/X_test_sparse.pkl', 'wb') as X_test_sparse_pkl:
    pickle.dump(X_test_sparse, X_test_sparse_pkl)
with open('kaggle_data/train_target.pkl', 'wb') as train_target_pkl:
    pickle.dump(y, train_target_pkl)

**Создайте объекты sklearn.linear_model.SGDClassifier с логистической функцией потерь и с hinge loss (логистическая регрессия и линейный SVM соответственно) . Остальные параметры  по умолчанию, разве что alpha=0.00007, n_jobs=-1 никогда не помешает. Обучите  модели на выборке (X_train, y_train).**

In [31]:
def fit_SGD (Train, y, explain = "fit_SGD"):
    # Разобьем обучающую выборку на 2 части в пропорции 7/3
    X_train, X_valid, y_train, y_valid = train_test_split(Train, y, test_size=0.3, 
                                                     random_state=0, stratify=y)
    
    sgd_logit = SGDClassifier( alpha=0.00007, loss ='log', random_state = 0,  n_jobs = -1)
    sgd_svm  = SGDClassifier( alpha=0.00007, loss ='hinge', random_state=0,  n_jobs = -1)
    print('fit...')
    %time sgd_logit.fit(X_train, y_train)
    %time sgd_svm.fit(X_train, y_train)
    # Сделаем прогнозы на отложенной выборке (X_valid, y_valid)
    print('predict...')
    pred_log = sgd_logit.predict(X_valid)
    pred_svm = sgd_svm.predict(X_valid)
    print (explain)
    print('log',accuracy_score(y_valid, pred_log))
    print('svm',accuracy_score(y_valid, pred_svm))
    return sgd_logit, sgd_svm

**обучим класификаторы  по признакам site + day + hour**

In [32]:
sgd_log, sgd_svm = fit_SGD(X_train_sparse, y, 'site + time')

fit...
Wall time: 34.1 s
Wall time: 26.4 s
predict...
site + time
('log', 0.35812701077073716)
('svm', 0.33224926563155688)


**сделаем класификатор на временных признаках**

In [33]:
X_train_test_sparse_time = hstack((session_timespan_encode,start_hour_encod, day_of_week_encod, tr_ts_df_time_encode))
X_train_test_sparse_time.shape

(136496, 5219)

In [34]:
X_train_time = X_train_test_sparse_time.tocsr()[: 95319, :]
X_test_time = X_train_test_sparse_time.tocsr()[95319 :, :]

**обучим класификаторы по признакам diff time+ session time + day + hour**

In [35]:
sgd_log_time, sgd_svm_time = fit_SGD(X_train_time, y, 'diff time+ session time + day + hour')

fit...
Wall time: 36.6 s
Wall time: 27.5 s
predict...
diff time+ session time + day + hour
('log', 0.078052874527906005)
('svm', 0.034760106308574623)


**точность классификатора очень низкое**

**составим из ответов этого класификатора и первого новые признаки и обучим на них новую модель**

In [36]:
log_pred_time = sgd_log_time.predict_proba(X_train_time) # предсказание временной модели
log_pred = sgd_log.predict_proba(X_train_sparse) # предсказание первой модели

In [38]:
X_train_add = np.hstack((log_pred, log_pred_time) )
X_train_add.shape

(95319L, 1100L)

**обучим класификаторы  по признакам first.model + time.model**

In [39]:
sgd_log_add, sgd_svm_add = fit_SGD(X_train_add, y, 'first.model + time.model')

fit...


KeyboardInterrupt: 

**точность улучшилась по сравнению с sites+ day + hour  но не значительно**

- загрузим словарь сайтов

In [32]:
site_diction = pd.read_csv('kaggle_data/site_indexes.txt', names =['key','site'])
site_diction.head()

,key,site
0,1,fr.openclassrooms.com
1,2,sigayret.fr
2,3,c1.adform.net
3,4,dnn506yrbagrg.cloudfront.net
4,5,ocsp.verisign.com


- создадим дополнительные признаки по первому и второму доменному именю

In [33]:
site_diction['_second'] = site_diction.site.apply(lambda x: '.'.join(x.split('.')[-2:]))
site_diction['_first'] = site_diction.site.apply(lambda x: x.split('.')[-1])
site_diction.astype(str).head()

,key,site,_second,_first
0,1,fr.openclassrooms.com,openclassrooms.com,com
1,2,sigayret.fr,sigayret.fr,fr
2,3,c1.adform.net,adform.net,net
3,4,dnn506yrbagrg.cloudfront.net,cloudfront.net,net
4,5,ocsp.verisign.com,verisign.com,com


- преобразуем в словари

In [34]:
site_diction = site_diction.astype(str)
second_dic = site_diction.set_index('key')['_second'].to_dict()
first_dic = site_diction.set_index('key')['_first'].to_dict()
second_dic['0'] = ''
first_dic['0'] =''

#### используем для создания признаков инструменты из text features

In [35]:
df_train = train_df[['site1', 'site2', 'site3', 'site4','site5', 
                    'site6','site7', 'site8', 'site9', 'site10']].fillna(0).astype(int).astype(str)

In [36]:
df_test = test_df[['site1', 'site2', 'site3', 
                                     'site4','site5', 
                                     'site6','site7', 'site8', 
                                     'site9', 'site10']].fillna(0).astype(int).astype(str)

In [37]:
df_train = df_train.replace('0','')
df_test = df_test.replace('0','')

In [38]:
df_train_str = df_train.apply(lambda x: ' '.join(x), axis =1)

In [39]:
df_test_str = df_test.apply(lambda x: ' '.join(x), axis =1)

In [40]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
cvt = CountVectorizer( ngram_range = (1,2), min_df =2)
tvt = TfidfVectorizer( ngram_range = (1,2), min_df =2)

In [41]:
y = train_df['user_id'].values

- создадим несколько таблиц признаков с разными значениями

In [42]:
def ngramm_analys (Train, Test, ngramm = (1,1) , min_freq = 1, analyzer = u'word'):
    cvt = CountVectorizer( ngram_range = ngramm, min_df =min_freq, analyzer = analyzer)
    tvt = TfidfVectorizer( ngram_range = ngramm, min_df =min_freq, analyzer = analyzer)
    X_train_cvt = cvt.fit_transform(Train.values)
    X_train_tvt = tvt.fit_transform(Train.values)
    X_test_cvt  = cvt.transform(Test.values)
    X_test_tvt  = tvt.transform(Test.values)
    return X_train_cvt, X_train_tvt, X_test_cvt, X_test_tvt

**обучим класификаторы по этим признакам** 

In [43]:
dic_test ={}
dic_cvt ={}
dic_tvt ={}
for ngramm, min_df in [((1,1),2) ,((1,2),2), ((2,2),2), ((2,3),2)]:
    print ('ngramm =', ngramm, '  min_fr =', min_df,'  analys = word')
    X_train_cvt, X_train_tvt, X_test_cvt, X_test_tvt =\
    ngramm_analys (df_train_str, df_test_str, ngramm  , min_df)
            
    key = str(ngramm) +" "+ str(min_df) +" word"
    #dic_test[key] = (X_test_cvt, X_test_tvt)
            
    sgd_log_ngramm, sgd_svm_ngramm = fit_SGD(X_train_cvt, y, 'cvt  '+ key)
    #dic_cvt[key] =(sgd_log_ngramm, sgd_svm_ngramm)
            
    sgd_log_ngramm, sgd_svm_ngramm = fit_SGD(X_train_tvt, y, 'tvt  '+ key)
    #dic_tvt[key] =(sgd_log_ngramm, sgd_svm_ngramm)    

('ngramm =', (1, 1), '  min_fr =', 2, '  analys = word')
fit...
Wall time: 43.2 s
Wall time: 43.1 s
predict...
cvt  (1, 1) 2 word
('log', 0.28815218911735907)
('svm', 0.26814939152328998)
fit...
Wall time: 45.6 s
Wall time: 25.3 s
predict...
tvt  (1, 1) 2 word
('log', 0.21307175828787242)
('svm', 0.29724436984193592)
('ngramm =', (1, 2), '  min_fr =', 2, '  analys = word')
fit...
Wall time: 55.9 s
Wall time: 50.7 s
predict...
cvt  (1, 2) 2 word
('log', 0.29353755770037765)
('svm', 0.27885018883759966)
fit...
Wall time: 1min 6s
Wall time: 51.5 s
predict...
tvt  (1, 2) 2 word
('log', 0.19390823891453352)
('svm', 0.30574206182682895)
('ngramm =', (2, 2), '  min_fr =', 2, '  analys = word')
fit...
Wall time: 50.7 s
Wall time: 35.4 s
predict...
cvt  (2, 2) 2 word
('log', 0.22870331514897188)
('svm', 0.24332074416002239)
fit...
Wall time: 42.5 s
Wall time: 55.8 s
predict...
tvt  (2, 2) 2 word
('log', 0.13480906420478389)
('svm', 0.26335851167995522)
('ngramm =', (2, 3), '  min_fr =', 2, '  a

-выберем наиболее сильные признаки - ngramm =(1,2) min_df = 2 для log CountVectorizer() сделаем предсказание и будем использовать его в следующем классификаторе

In [49]:
X_train_cvt, X_train_tvt, X_test_cvt, X_test_tvt =\
    ngramm_analys (df_train_str, df_test_str, (1,2) ,  min_freq = 2)
    

In [50]:
sgd_log_ngramm, sgd_svm_ngramm = fit_SGD(X_train_cvt, y, 'cvt  ngramm=(1,2) min_df = 2')

fit...
Wall time: 42.5 s
Wall time: 31.5 s
predict...
cvt  ngramm=(1,2) min_df = 2
('log', 0.29353755770037765)
('svm', 0.27885018883759966)


In [55]:
pred_cvt = sgd_log_ngramm.predict_proba(X_train_cvt)

In [60]:
from scipy.sparse import hstack
X_train_new_add = hstack((X_train_tvt, csr_matrix(pred_cvt)))

In [61]:
sgd_log_ngramm, sgd_svm_ngramm = fit_SGD(X_train_new_add, y, 'tvt + model.cvt   ngramm=(1,2) min_df = 2')

fit...
Wall time: 6min 37s
Wall time: 4min 45s
predict...
tvt + model.cvt   ngramm=(1,2) min_df = 2
('log', 0.25171352636732408)
('svm', 0.3153937613652259)


- добавим предсказание временной модели к данным

In [ ]:
X_train_new_add = hstack((X_train_tvt, csr_matrix(pred_cvt), csr_matrix(logit_valid_pred_T))) #предсказание временной модели

**добавим в качестве признаков первое и второе доменное имя**

In [45]:
df_train = train_df[['site1', 'site2', 'site3', 'site4','site5', 
                    'site6','site7', 'site8', 'site9', 'site10']].fillna(0).astype(int).astype('str')

In [46]:
df_test = test_df[['site1', 'site2', 'site3', 
                                     'site4','site5', 
                                     'site6','site7', 'site8', 
                                     'site9', 'site10']].fillna(0).astype(int).astype('str')

In [47]:
df2 = df_train.applymap(lambda x: second_dic[x])
df3 = df_train.applymap(lambda x: first_dic[x])
df3.head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
1,com,com,com,fr,com,com,net,com,net,com
2,com,com,com,com,com,com,com,com,com,com
3,com,fr,io,com,com,fr,fr,com,com,fr
4,com,com,com,com,com,com,com,com,com,com
5,org,org,org,org,org,fr,fr,org,org,org


In [27]:
df2_test = df_test.applymap(lambda x: second_dic[x])
df3_test = df_test.applymap(lambda x: first_dic[x])

In [29]:
df_train2 = df_train2.apply(lambda x: ' '.join(x) if x != '0' else '', axis =1)
df_test2 = df_test2.apply(lambda x: ' '.join(x), axis =1)

In [30]:
df2 = df2.apply(lambda x: ' '.join(x), axis =1)
df2_test = df2_test.apply(lambda x: ' '.join(x), axis =1)
df3 = df3.apply(lambda x: ' '.join(x), axis =1)
df3_test = df3_test.apply(lambda x: ' '.join(x), axis =1)

In [31]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
cvt = CountVectorizer( ngram_range = (1,2), min_df =2)#, analyzer=u'char')
tvt = TfidfVectorizer( ngram_range = (1,2), min_df =2)#, analyzer=u'char')

In [32]:
%time X_train_ngram_c = cvt.fit_transform(df_train2.values)
X_train_ngram_t = tvt.fit_transform(df_train2.values)

Wall time: 6.49 s


In [35]:
%time X_test_ngram_c = cvt.transform(df_test2.values)
X_test_ngram_t = tvt.transform(df_test2.values)

Wall time: 2.16 s


In [36]:
X_train_ngram_c.shape, X_train_ngram_t.shape

((95319, 60252), (95319, 60252))

In [37]:
y = train_df['user_id'].values

In [59]:
%%time
cvt = CountVectorizer()
tvt = TfidfVectorizer()
X_train_ngram_c2 = cvt.fit_transform(df2.values)
X_train_ngram_t2 = tvt.fit_transform(df2.values)
X_test_ngram_c2 = cvt.transform(df2_test.values)
X_test_ngram_t2 = tvt.transform(df2_test.values)

Wall time: 10.7 s


In [60]:
X_train_ngram_c2.shape, X_train_ngram_t2.shape

((95319, 9710), (95319, 9710))

In [61]:
%%time
cvt = CountVectorizer()#, analyzer=u'char')
tvt = TfidfVectorizer()#, analyzer=u'char')
X_train_ngram_c3 = cvt.fit_transform(df3.values)
X_train_ngram_t3 = tvt.fit_transform(df3.values)
X_test_ngram_c3 = cvt.transform(df3_test.values)
X_test_ngram_t3 = tvt.transform(df3_test.values)

Wall time: 6.34 s


In [62]:
X_train_ngram_c3.shape, X_train_ngram_t3.shape

((95319, 144), (95319, 144))

In [63]:
from scipy.sparse import hstack
X_train_new_c = hstack((X_train_ngram_c, X_train_ngram_c2, X_train_ngram_c3))
X_train_new_t = hstack((X_train_ngram_t, X_train_ngram_t2, X_train_ngram_t3))
X_train_new_c.shape, X_train_new_t.shape

((95319, 70106), (95319, 70106))

**обучим те же класификаторы что и до этого с преобразованием CountVectorizer**

In [111]:
sgd_log_new_c, sgd_svm_new_c = fit_SGD(X_train_new_c, y, 'cvt ngramm + dominate names')

fit...
Wall time: 55.8 s
Wall time: 37.1 s
predict...
cvt ngramm + dominate names
('log', 0.2470625262274444)
('svm', 0.23080151070079732)


**обучим те же класификаторы что и до этого с преобразованием TfidfVectorizer**

In [110]:
sgd_log_new_t, sgd_svm_new_t = fit_SGD(X_train_new_t, y, 'tvt ngramm + dominate names')

fit...
Wall time: 46.7 s
Wall time: 35.3 s
predict...
tvt ngramm + dominate names
('log', 0.23569730032172331)
('svm', 0.30661630997342287)


In [112]:
from scipy.sparse import hstack
X_train_new2_c = hstack((X_train_ngram_c, X_train_ngram_c2))
X_train_new2_t = hstack((X_train_ngram_t, X_train_ngram_t2))
X_train_new_c.shape, X_train_new_t.shape

((95319, 70106), (95319, 70106))

**обучим те же класификаторы что и до этого с преобразованием CountVectorizer**

In [113]:
sgd_log_new2_c, sgd_svm_new2_c = fit_SGD(X_train_new2_c, y, 'cvt ngramm + 1 dominate names')

fit...
Wall time: 50.2 s
Wall time: 37.3 s
predict...
cvt ngramm + 1 dominate names
('log', 0.26409288012309412)
('svm', 0.24422996223248006)


**обучим те же класификаторы что и до этого с преобразованием TfidfVectorizer**

In [114]:
sgd_log_new2_t, sgd_svm_new2_t = fit_SGD(X_train_new2_t, y, 'tvt ngramm + 1 dominate names')

fit...
Wall time: 58 s
Wall time: 43.3 s
predict...
tvt ngramm + 1 dominate names
('log', 0.24674779689467058)
('svm', 0.31095258078052873)


**добавим временные признаки к ngramm**

In [103]:
start_hour_encod.shape, day_of_week_encod.shape

((136496, 17), (136496, 7))

In [104]:
start_hour_train = start_hour_encod.tocsr()[: 95319, :]
start_hour_test = start_hour_encod.tocsr()[95319 :, :]
day_of_week_train = day_of_week_encod.tocsr()[: 95319, :]
day_of_week_test = day_of_week_encod.tocsr()[95319 :, :]

In [105]:

X_train_time_c = hstack((X_train_ngram_c, start_hour_train, day_of_week_train))
X_train_time_t = hstack((X_train_ngram_t, start_hour_train, day_of_week_train))
X_train_time_c.shape, X_train_time_t.shape

((95319, 60276), (95319, 60276))

**обучим те же класификаторы что и до этого с преобразованием CountVectorizer**

In [108]:
sgd_log_time, sgd_svm_time = fit_SGD(X_train_time_c, y, 'cvt ngramm + time')

fit...
Wall time: 41.3 s
Wall time: 32.8 s
predict...
cvt ngramm + time
('log', 0.35798713106728214)
('svm', 0.33676038606798153)


**обучим те же класификаторы что и до этого с преобразованием TfidfVectorizer**

In [109]:
sgd_log_time2, sgd_svm_time2 = fit_SGD(X_train_time_t, y, 'tvt ngramm + time')

fit...
Wall time: 46.3 s
Wall time: 35.5 s
predict...
tvt ngramm + time
('log', 0.25884739124353057)
('svm', 0.36071478528465517)


In [82]:
X_train_new = hstack((X_train_sparse, X_train_ngram))
X_test_new = hstack((X_test_sparse, X_test_ngram))

In [20]:
from sklearn.model_selection import GridSearchCV

In [21]:
parameters = {'penalty': ('l2', 'l1'), 
              'alpha': [0.00015, 0.00012, 0.00008, 0.0001]
              }           
parameters = {'alpha': [0.00007, 0.00006, 0.00009, 0.00012, 0.00008, 0.0001]
              }           

In [22]:
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=7)

In [23]:
clf = GridSearchCV(sgd_svm, parameters, cv =skf)

In [24]:
%time clf.fit(X_train, y_train)

Wall time: 19min 49s


GridSearchCV(cv=StratifiedKFold(n_splits=4, random_state=7, shuffle=True),
       error_score='raise',
       estimator=SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=-1,
       penalty='l2', power_t=0.5, random_state=7, shuffle=True, verbose=0,
       warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'penalty': ('l2', 'l1'), 'alpha': [0.00015, 0.00012, 8e-05, 0.0001]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [25]:
clf.best_score_, clf.best_params_

(0.2875919847728669, {'alpha': 8e-05, 'penalty': 'l2'})

In [31]:
accuracy_score(y_valid, clf.best_estimator_ .predict(X_valid))

0.30196530983354314

In [53]:
clf.best_score_, clf.best_params_

(0.28089264571437134, {'alpha': 0.0001, 'penalty': 'l2'})

**Запишите в файл *answer5_2.txt* через пробел доли правильных ответов логистической регресии и линейного SVM, обученных с помощью стохастического градиентного спуска, на отложенной выборке. Округлите до 3 знаков после разделителя. Полученный файл будет ответом на 2 вопрос теста.**

In [32]:
write_answer_to_file('{}  {}'.format(round(accuracy_score(y_valid, logit_valid_pred), 3),
                                    round(accuracy_score(y_valid, svm_valid_pred), 3)),
                     'answer5_2.txt')

**Сделайте прогноз для тестовой выборки с помощью sgd_logit.**

In [90]:
logit_test_pred = sgd_logit.predict(X_test_new)

In [26]:
logit_test_pred = clf.best_estimator_ .predict(X_test_ngram)

**Запишите ответы в файл и сделайте посылку на Kaggle. Далее будет короткое Peer-Review задание, в котором надо проверить посылки друг друга на Kaggle. Поэтому дайте своей команде (из одного человека) на Kaggle говорящее название – по шаблону "[YDF & MIPT] Coursera Username", чтоб можно было идентифицировать Вашу посылку на [лидерборде](https://inclass.kaggle.com/c/identify-me-if-you-can-yandex-mipt/leaderboard).**

**Результат, который мы только что получили, соответствует бейзлайну "SGDCLassifer" на лидерборде, задача на эту неделю – как минимум его побить, дополнительные баллы будут для тех, кто попадет в топ-10 и топ-30 по итогам этой недели.**

In [91]:
def write_to_submission_file(predicted_labels, out_file,
                             target='user_id', index_label="session_id"):
    # turn predictions into data frame and save as csv file
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [92]:
write_to_submission_file (logit_test_pred, 'kaggle_data/[YDF&MIPT]_Coursera_Oleg67.csv')

## Пути улучшения
На этой неделе дается много времени на соревнование. Не забывайте вносить хорошие идеи, к которым Вы пришли по ходу соревнования, в финальный проект (.pdf или .ipynb).
Что можно попробовать:
 - Использовать ранее построенные признаки для улучшения модели (проверить их можно на меньшей выборке по 150 пользователям – это быстрее)
 - Настроить параметры моделей (например, коэффициенты регуляризации)
 - Если позволяют мощности (или хватает терпения), можно попробовать смешивание (блендинг) ответов бустинга и линейной модели. [Вот](http://mlwave.com/kaggle-ensembling-guide/) один из самых известных тьюториалов по смешиванию ответов алгоритмов
 - Обратите внимание, что в соревновании также даны исходные данные о посещенных 550 пользователями веб-страницах (550 csv-файлов в *train.zip*). По этим данным можно сформировать свою обучающую выборку. 

На 6 неделе мы пройдем тьюториал по Vowpal Wabbit и попробуем его в деле – в соревновании.

In [41]:
df_train1 = train_df[['site1', 'site2', 'site3', 
                                     'site4','site5', 
                                     'site6','site7', 'site8', 
                                     'site9', 'site10']].fillna(0).astype('str')

In [ ]:
from sklearn.cluster import DBSCAN
clu = DBSCAN(eps=0.6, min_samples=500 )
%time clus = clu.fit_predict(matrix_time)

In [ ]:
np.unique(clus, return_counts =True)

In [45]:
from sklearn.mixture import BayesianGaussianMixture
BGM = BayesianGaussianMixture(n_components =20)
%time BGM.fit(df_train1.values)

Wall time: 3min 1s


C:\Users\User\Anaconda2\lib\site-packages\sklearn\mixture\base.py:237: ConvergenceWarning: Initialization 1 did not converged. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarning)


BayesianGaussianMixture(covariance_prior=None, covariance_type='full',
            degrees_of_freedom_prior=None, init_params='kmeans',
            max_iter=100, mean_precision_prior=None, mean_prior=None,
            n_components=20, n_init=1, random_state=None, reg_covar=1e-06,
            tol=0.001, verbose=0, verbose_interval=10, warm_start=False,
            weight_concentration_prior=None,
            weight_concentration_prior_type='dirichlet_process')

In [57]:
lables = BGM.predict(df_train1.values)

In [58]:
np.unique(lables, return_counts =True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19], dtype=int64),
 array([12802,  3332,  3239,  8405,  2428,  3098,  2490,  5426,  2067,
         3414,  2800,  6209, 16916,  2866,  5041,  2243,  2183,  2292,
         5109,  2959], dtype=int64))